# [Aligning Big Brains and Atlases](https://biop.github.io/ijp-imagetoatlas/) in Python

This series of notebook demoes the use of ABBA with python.

You need to have executed the first notebook in order to re-open its registration result.

This notebook shows how to convert the pixel position of a slice to its coordinates in the atlas (forth and back).


In [1]:
# core dependencies
import os
import time
from pathlib import Path


from bg_atlasapi import show_atlases
from bg_atlasapi import utils

from abba_python.abba import Abba

In [2]:
# -- NOT HEADLESS
aligner = Abba('Adult Mouse Brain - Allen Brain Atlas V3p1') # You may have to click a window!
aligner.show_bdv_ui()  # creates and show a bdv view

save_dir = os.path.join(os.getcwd(), 'temp', 'notebook0', 'state') # execute the first notebook to get this file!
aligner.state_load(save_dir+"/state.abba") # full absolute path needed

# ALL COMMANDS ARE PERFORMED ON SELECTED SLICES!!
# since we want to register all of them, we select all of them
aligner.select_all_slices()

# change display settings : channel index, min, max
aligner.change_display_settings(0, 0, 500)
aligner.change_display_settings(1, 0, 1200)

# YOU MAY NEED TO CLICK ON THE UI TO FINISH THE EXECUTION OF THIS CELL (angle change notification)

[java.lang.Enum.toString] Spimdata mpicbg.spim.data.SpimData@3c8e4666 is using the Bdv Playground global cache.
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: ba [ch.epfl.biop.atlas.mouse.allen.ccfv3p1.command.AllenBrainAdultMouseAtlasCCF2017v3p1Command]


- Main Java Dependencies:
['net.imagej:imagej:2.14.0', 'net.imagej:imagej-legacy:1.2.1', 'ch.epfl.biop:ImageToAtlasRegister:0.8.0']
- Atlas cache folder:
C:\Users\chiarutt\cached_atlas
- Elastix path:
C:\Users\chiarutt\AppData\Local\abba-python-0.8.0\win\elastix-5.0.1-win64\elastix.exe
- Transformix path:
C:\Users\chiarutt\AppData\Local\abba-python-0.8.0\win\elastix-5.0.1-win64\transformix.exe
- DeepSlice conda environment:
C:\Users\chiarutt\AppData\Local\abba-python-0.8.0\envs\deepslice v1.1.5


[java.lang.Enum.toString] Creating MultiSlicePositioner instance
[java.lang.Enum.toString] MultiSlicePositioner instance created
[java.lang.Enum.toString] [WARNING] Ignoring unsupported output: mp [ch.epfl.biop.atlas.aligner.MultiSlicePositioner]
[java.lang.Enum.toString] Keymap list file C:\Users\chiarutt\AppData\Roaming\fiji\bigdataviewer\config\keymaps\keymaps.yaml not found. Using builtin styles.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Appearance settings file C:\Users\chiarutt\AppData\Roaming\fiji\bigdataviewer\config/appearance.yaml not found. Using defaults.[java.lang.Enum.toString] 
[java.lang.Enum.toString] Registration plugin RegisterSlicesElastixAffineCommand discovered
[java.lang.Enum.toString] Registration plugin RegisterSlicesBigWarpCommand discovered
[java.lang.Enum.toString] Registration plugin MirrorDoCommand discovered
[java.lang.Enum.toString] Registration plugin RegisterSlicesElastixSplineCommand discovered
[java.lang.Enum.toString] IRunTimeClassAdapters

In [3]:
# programmatic way to show (or hide) sections and channels
aligner.get_bdv_view().setSelectedSlicesVisibility(True)
aligner.get_bdv_view().setSelectedSlicesVisibility(0, True)

In [4]:
mp = aligner.mp
mp.selectSlice(mp.getSlices()) # select all slices

In [5]:
mp.deselectSlice(mp.getSlices()) # deselect all

In [6]:
mp.selectSlice(mp.getSlices().get(2)) # select the third slice

In [7]:
# The slices are always sorted from small z to high z. To keep track of who's who, reference them before moving them
slice_0 = mp.getSlices().get(0) 
slice_1 = mp.getSlices().get(1)
slice_2 = mp.getSlices().get(2)

In [8]:

from jpype.types import JString, JArray, JDouble

# Get transformation
transform_pix_to_atlas = slice_0.getSlicePixToCCFRealTransform()

DoubleArray = JArray(JDouble)

coordInImage = DoubleArray(3)
coordInCCF = DoubleArray(3)

coordInImage[0] = 500 # X (pixel)
coordInImage[1] = 500 # Y
coordInImage[2] = 0 # Z

transform_pix_to_atlas.inverse().apply(coordInImage,coordInCCF)

print('CCF coord (mm):'+str(coordInCCF)) # It may be printed on a cell above with jupyter lab


CCF coord (mm):[3.57824625991771, 7.646480330904136, 4.099084106904822]


In [9]:
# We can also find the coordinates of the atlas into a slice
# a coordinate in z far from 0, means that it's not in the slice, but far out
# See https://forum.image.sc/t/coordinates-in-slice-to-atlas-using-abba-python/79513 

coordInCCF = DoubleArray(3)
coordInCCF[0] = 6.60 # X (pixel)
coordInCCF[1] = 4.31 # Y
coordInCCF[2] = 5.58 # Z

coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))
    


Slice[0] coord (pixel):[355.3614572699433, 220.97872475513975, 3410.938448709107]
Slice[1] coord (pixel):[353.0728632584608, 284.72379308011347, 2841.880551661362]
Slice[2] coord (pixel):[393.48660226304787, 328.0700217403849, 2272.8406319846727]
Slice[3] coord (pixel):[425.27774678379876, 303.74450601793967, 1703.8106641519387]
Slice[4] coord (pixel):[450.0615274639213, 306.1193688335815, 1134.7577094551048]
Slice[5] coord (pixel):[465.4176142951741, 330.4015782242297, 565.7167152546805]
Slice[6] coord (pixel):[479.34354545334685, 342.6166997379125, -3.3130987100138394]
Slice[7] coord (pixel):[461.2622501037235, 314.05686984978075, -572.3599546856909]
Slice[8] coord (pixel):[500.12716093664744, 359.2511040827763, -1141.3938991062623]
Slice[9] coord (pixel):[477.29238585861776, 349.246218551061, -1710.4492048368145]
Slice[10] coord (pixel):[502.3989499605028, 354.35216426935534, -2279.4874603481126]
Slice[11] coord (pixel):[484.9440859360184, 361.81501063956375, -2848.5325780635458]
Sl

In [10]:
# The slice indexed 7 should be the one which is the nearest to the atlas point (Lowest absolute value in Z)

In [11]:
# You can normalize the slices sections by expanding them to cover the space between neighboring slices, no more, no less
aligner.select_all_slices()
aligner.set_slices_thickness_match_neighbors() # not critical, but for 3d reconstruction it will allow for each slice to occupy the place available between its neighbors
# The coordinates will be different in Z
coordInImage = DoubleArray(3)

for idx in range(0,mp.getSlices().size()):
    # Get transformation
    transform_pix_to_atlas = mp.getSlices().get(idx).getSlicePixToCCFRealTransform()
    transform_pix_to_atlas.apply(coordInCCF, coordInImage)
    print('Slice['+str(idx)+'] coord (pixel):'+str(coordInImage))

Slice[0] coord (pixel):[355.3614572699433, 220.97872475513975, 5.9940095136275735]
Slice[1] coord (pixel):[353.0728632584608, 284.72379308011347, 4.9940962968954095]
Slice[2] coord (pixel):[393.48660226304787, 328.0700217403849, 3.994206585888334]
Slice[3] coord (pixel):[425.27774678379876, 303.74450601793967, 2.9941665650318994]
Slice[4] coord (pixel):[450.0615274639213, 306.1193688335815, 1.9941424004208481]
Slice[5] coord (pixel):[465.4176142951741, 330.4015782242297, 0.9941727812625423]
Slice[6] coord (pixel):[479.34354545334685, 342.6166997379125, -0.005829764022724156]
Slice[7] coord (pixel):[461.2622501037235, 314.05686984978075, -1.0058279282062157]
Slice[8] coord (pixel):[500.12716093664744, 359.2511040827763, -2.005816748171893]
Slice[9] coord (pixel):[477.29238585861776, 349.246218551061, -3.0058072561549607]
Slice[10] coord (pixel):[502.3989499605028, 354.35216426935534, -4.005838639811199]
Slice[11] coord (pixel):[484.9440859360184, 361.81501063956375, -5.005843471354506]
